In [1]:
from google.colab import drive 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/PL

/content/drive/MyDrive/PL


In [3]:
# !pip install cmath
!pip install torch
!pip install sklearn
!pip install pandas
!pip install librosa
!pip install numpy
!pip install speechpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=b8b83ced0d0e73cc5bde84ab0a121868dfda189c045e49e1d63e7fd4374b2783
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install transformers
!pip install jiwer
!pip install pyctcdecode
from pyctcdecode import build_ctcdecoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 76.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 KB 20.7 MB/s eta 0:00:00


In [ ]:
# from sklearn.model_selection import train_test_split
# import pandas as pd
# data = pd.read_csv("/content/drive/MyDrive/PL/metadata_train.csv")

# train,test = train_test_split(data, test_size=0.1, random_state=42)

In [ ]:
# train.to_csv("/content/drive/MyDrive/PL/train.csv")
# test.to_csv("/content/drive/MyDrive/PL/dev.csv")

In [7]:
from cmath import acos
import torch
from torch.utils.data import Dataset
# import matplotlib.pyplot as plt
import os
import pandas as pd
# import numpy as np
# from torch.utils.data import DataLoader
# from infer import phonetic_embedding
from help import Atention, wav_norm
from char_embedding import tensor_to_text,text_to_tensor
import librosa
import numpy as np

data = pd.read_csv("train.csv")
# print(data['Wav_path'])
sample = data.shape[0]
# cols = ['Wav_path', 'Lyric_path']

class MDD_Dataset(Dataset):

    def __init__(self):
        acoustic_canonical = data
        self.n_samples = sample
        A = acoustic_canonical['Wav_path']
        C = acoustic_canonical['Lyric_path']
        B = acoustic_canonical['Lyric_path'] #output
        

        self.A_data = A 
        self.C_data = C
        self.y_data = B 

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        p = self.A_data[index]
        p = p.split(".")[0]
        base_dir = '/content/drive/MyDrive/vocal_song_phonetic/'
        p = p + str(".npy")
        p = np.load(base_dir + str(p))
        phonetic = torch.tensor(p)
       
        linguistic = text_to_tensor(self.C_data[index])
        linguistic = torch.tensor(linguistic)
        label = text_to_tensor(self.y_data[index])
        label = torch.tensor(label)
        return phonetic, linguistic, label

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples



In [ ]:
# from model import Net
# from cProfile import label
# from infer import phonetic_embedding
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import torch.nn as nn
import torch.nn.functional as F
from phonetic_encoder import Phonetic_encoder
from acoustic_encoder import Acoustic_encoder
from linguistic_encoder import Linguistic_encoder
from char_embedding import text_to_tensor, clean_corpus
from help import wav_norm, Atention
import numpy as np
# from help import beam_search_decoding
from model import Acoustic_Phonetic_Linguistic
# from dataloader import MDD_Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
from jiwer import wer, cer
tokenizer = Wav2Vec2Processor.from_pretrained("pretrained_finetuned")

"""
"""
f = open("/content/drive/MyDrive/PL/vocal_loss.txt", 'a')
MDD = MDD_Dataset()
# print(data)
net = Acoustic_Phonetic_Linguistic()
net.to('cuda')

dev = pd.read_csv("dev.csv")
# net = torch.load('/content/drive/MyDrive/PL/raw_checkpoint.pth14')
# net = net.to('cpu')
train_loader = DataLoader(dataset=MDD,
                          batch_size=1,
                          shuffle=True,
                          num_workers=0)



# convert to an iterator and look at one random sample


ctc_loss = nn.CTCLoss(blank = 95)
# optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)
optimizer = optim.AdamW(net.parameters(), lr = 0.00001)
# optimizer = optim.SGD(net.parameters(), 0.01, momentum = 0.9)
# optimizer = torch.load('/home/tuht/train_wav2vec/MDD_Checkpoint/checkpoint_optim.pth')
best_WER = 100
for epoch in range(50):  # loop over the dataset multiple times
    net.train().to('cuda')
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        phonetic, linguistic, labels = data
        # acoustic = torch.tensor(0).to('cuda')
        phonetic = phonetic.to('cuda')
        linguistic = linguistic.to('cuda')
        labels = labels.to('cuda')
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(phonetic, linguistic)
        outputs = outputs.unsqueeze(1)
        input_lengths = outputs.shape
        target_lengths = labels.shape
        target = labels
        input_lengths = [input_lengths[0]]
        target_lengths =[target_lengths[1]]
        input_lengths = torch.tensor(input_lengths)
        target_lengths = torch.tensor(target_lengths)
        outputs = (F.log_softmax(outputs, dim=2))
        loss = ctc_loss(outputs, labels, input_lengths, target_lengths)
        # print(i)
        # print(loss)
        f.write("(" +str(epoch) + "," + str(i) + ")" + "  loss: " + str(loss) + "\n") 
        loss.backward()
        optimizer.step()
    WER = 0
    net.eval().to('cuda')
    for i in range(len(dev)):
        p = dev['Wav_path'][i]
        p = p.split(".")[0]
        base_dir = '/content/drive/MyDrive/vocal_song_phonetic/'
        p = p + str(".npy")
        p = np.load(base_dir + str(p))
        phonetic = torch.tensor(p)
       
        linguistic = text_to_tensor(dev['Lyric_path'][i])
        linguistic = torch.tensor(linguistic)
        label = text_to_tensor(dev['Lyric_path'][i])
        label = torch.tensor(label)
        phonetic = phonetic.to('cuda')
        linguistic = linguistic.to('cuda')
        phonetic = phonetic.unsqueeze(0)
        linguistic = linguistic.unsqueeze(0)
        outputs = net(phonetic, linguistic)
        outputs = outputs.unsqueeze(0)
        x = F.log_softmax(outputs,dim=2)
        labels = ['ắ', 'ồ', 'z', 'ứ', 'ỡ', 'ì', 'x', 'ặ', 'u', 'ẹ', 'd', 'ỵ', 'r', 'p', 't', 'ỳ', 'ẩ', 'f', 'ó', 'á', 'v', 'ã', 'i', 'ư', 'ở', 'ễ', 'ụ', 'ú', 'ũ', ' ', 'ă', 'é', 'ằ', 'a', 'ấ', 'ờ', 'ữ', 'ớ', 'n', 'ý', 's', 'h', 'ơ', 'ị', 'l', 'c', 'k', 'ỷ', 'ỗ', 'ế', 'ẻ', 'ợ', 'ẫ', 'í', 'ỏ', 'ủ', 'g', 'q', 'j', 'ò', 'ỹ', 'ự', 'ô', 'b', 'y', 'ĩ', 'ỉ', 'ẵ', 'ầ', 'ê', 'ộ', 'ậ', 'm', 'ń', 'o', 'ọ', 'đ', 'ẽ', 'ử', 'à', 'è', 'e', 'ẳ', 'ổ', 'ù', 'w', 'ả', 'ạ', 'â', 'ệ', 'ề', 'õ', 'ố', 'ể', 'ừ',]
        x = x.detach().cpu().numpy()
        decoder = build_ctcdecoder(
            labels = labels,
            
        )
        x = x.squeeze(0)
        ground_truth = (clean_corpus(dev['Lyric_path'][i]))
        hypothesis = str(decoder.decode(x))
        error = cer(ground_truth, hypothesis)
        WER = WER + error
    print(epoch)
    print(WER)
    if WER/len(dev)<best_WER:
      print("save " + str(epoch))
      # print(epoch)
      # print(WER)
      best_WER = WER/len(dev)
      print(best_WER)
      torch.save(net, '/content/drive/MyDrive/PL/MDD_Checkpoint/vocal_checkpoint.pth')
        
print('Finished Training')


/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/content/drive/MyDrive/PL/linguistic_encoder.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float)


0
51.90985808747225
save 0
0.4897156423346439
1
46.68650132502654
save 1
0.4404386917455334
2
45.365100795686025
save 2
0.4279726490159059
3
45.05404392687716
save 3
0.4250381502535581
4
44.694133905946735
save 4
0.4216427726976107
5
44.57325048917713
save 5
0.4205023631054446
6
44.29809137715863
save 6
0.4179065224260248
7
44.28136764063459
save 7
0.41774875132674144
8
44.18679456902086
save 8
0.41685655253793263
9
43.96395431707662
save 9
0.4147542860101568
10
43.76067376546919
save 10
0.4128365449572565
11
44.000565851975004
12
44.179314302305315
13
43.59082420376029
save 13
0.4112341906015122
14
43.599868162697874
15
43.330021766920616
save 15
0.4087737902539681
16
43.48002122245216
17
43.712386581049316
18
43.54815115994965
19
43.43260320076995
20
43.74174828257709
21
43.35992363368725
22
43.280467586419526
save 22
0.4083062979850899
23
43.26112317357824
save 23
0.40812380352432304
24
43.759810726054106
25
43.1824265399046
save 25
0.4073813824519302
26
43.808792786901556
27
43.350

In [8]:
# from model import Net
# from cProfile import label
# from infer import phonetic_embedding
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import torch.nn as nn
import torch.nn.functional as F
from phonetic_encoder import Phonetic_encoder
from acoustic_encoder import Acoustic_encoder
from linguistic_encoder import Linguistic_encoder
from char_embedding import text_to_tensor, clean_corpus
from help import wav_norm, Atention
import numpy as np
# from help import beam_search_decoding
from model import Acoustic_Phonetic_Linguistic, Phonetic_Only
# from dataloader import MDD_Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
from jiwer import wer, cer
tokenizer = Wav2Vec2Processor.from_pretrained("pretrained_finetuned")

"""
"""
f = open("/content/drive/MyDrive/PL/vocal_loss.txt", 'a')
MDD = MDD_Dataset()
# print(data)
net = Phonetic_Only()
net.to('cuda')

dev = pd.read_csv("dev.csv")
# net = torch.load('/content/drive/MyDrive/PL/raw_checkpoint.pth14')
# net = net.to('cpu')
train_loader = DataLoader(dataset=MDD,
                          batch_size=1,
                          shuffle=True,
                          num_workers=0)



# convert to an iterator and look at one random sample


ctc_loss = nn.CTCLoss(blank = 95)
# optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)
optimizer = optim.AdamW(net.parameters(), lr = 0.001)
# optimizer = optim.SGD(net.parameters(), 0.01, momentum = 0.9)
# optimizer = torch.load('/home/tuht/train_wav2vec/MDD_Checkpoint/checkpoint_optim.pth')
best_WER = 100
for epoch in range(50):  # loop over the dataset multiple times
    net.train().to('cuda')
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        phonetic, linguistic, labels = data
        # acoustic = torch.tensor(0).to('cuda')
        phonetic = phonetic.to('cuda')
        labels = labels.to('cuda')
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(phonetic)
        outputs = outputs.unsqueeze(1)
        input_lengths = outputs.shape
        target_lengths = labels.shape
        target = labels
        input_lengths = [input_lengths[0]]
        target_lengths =[target_lengths[1]]
        input_lengths = torch.tensor(input_lengths)
        target_lengths = torch.tensor(target_lengths)
        outputs = (F.log_softmax(outputs, dim=2))
        loss = ctc_loss(outputs, labels, input_lengths, target_lengths)
        # print(i)
        # print(loss)
        f.write("(" +str(epoch) + "," + str(i) + ")" + "  loss: " + str(loss) + "\n") 
        loss.backward()
        optimizer.step()
    WER = 0
    net.eval().to('cuda')
    for i in range(len(dev)):
        p = dev['Wav_path'][i]
        p = p.split(".")[0]
        base_dir = '/content/drive/MyDrive/vocal_song_phonetic/'
        p = p + str(".npy")
        p = np.load(base_dir + str(p))
        phonetic = torch.tensor(p).to('cuda')
        phonetic = phonetic.unsqueeze(0)
        outputs = net(phonetic)
        outputs = outputs.unsqueeze(0)
        x = F.log_softmax(outputs,dim=2)
        labels = ['ắ', 'ồ', 'z', 'ứ', 'ỡ', 'ì', 'x', 'ặ', 'u', 'ẹ', 'd', 'ỵ', 'r', 'p', 't', 'ỳ', 'ẩ', 'f', 'ó', 'á', 'v', 'ã', 'i', 'ư', 'ở', 'ễ', 'ụ', 'ú', 'ũ', ' ', 'ă', 'é', 'ằ', 'a', 'ấ', 'ờ', 'ữ', 'ớ', 'n', 'ý', 's', 'h', 'ơ', 'ị', 'l', 'c', 'k', 'ỷ', 'ỗ', 'ế', 'ẻ', 'ợ', 'ẫ', 'í', 'ỏ', 'ủ', 'g', 'q', 'j', 'ò', 'ỹ', 'ự', 'ô', 'b', 'y', 'ĩ', 'ỉ', 'ẵ', 'ầ', 'ê', 'ộ', 'ậ', 'm', 'ń', 'o', 'ọ', 'đ', 'ẽ', 'ử', 'à', 'è', 'e', 'ẳ', 'ổ', 'ù', 'w', 'ả', 'ạ', 'â', 'ệ', 'ề', 'õ', 'ố', 'ể', 'ừ',]
        x = x.detach().cpu().numpy()
        decoder = build_ctcdecoder(
            labels = labels,
            
        )
        x = x.squeeze(0)
        ground_truth = (clean_corpus(dev['Lyric_path'][i]))
        hypothesis = str(decoder.decode(x))
        error = cer(ground_truth, hypothesis)
        WER = WER + error
    print(epoch)
    print(WER)
    if WER/len(dev)<best_WER:
      print("save " + str(epoch))
      # print(epoch)
      # print(WER)
      best_WER = WER/len(dev)
      print(best_WER)
      torch.save(net, '/content/drive/MyDrive/PL/MDD_Checkpoint/no_linguistic_vocal_checkpoint.pth')
        
print('Finished Training')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


0
96.42511009903221
save 0
0.9096708499908699
1
61.9815346063328
save 1
0.5847314585503094
2
46.05965181762243
save 2
0.4345250171473814
3
44.414971797417316
save 3
0.41900916790016335
4
44.478555915441774
5
44.56808397596326
6
43.85708390496426
save 6
0.41374607457513457
7
43.95150678645302
8
43.493756910568614
save 8
0.41031846142045864
9
44.050779141265295
10
44.035797826556355
11
43.700854789003586
12
43.455871197254545
save 12
0.40996104903070324
13
43.13841063541566
save 13
0.40696613806995907
14
43.3725868077413
15
43.4991077183926
16
43.51695331266012
17
44.23127775713887
18
43.305247410720874
19
43.2743587913094
20
43.016474484435
save 20
0.4058157970229717
21
43.13909512578376
22
43.34982950452361
23
43.45967200982931
24
43.41360431241227
25
43.20079826454078
26
43.52682939369963
27
43.23118431335593
28
43.72664369434233
29
43.136805792270096
30
43.53556120199259
31
43.61503763531481
32
43.37154670457896
33
43.904638832338044
34
43.065003333976186
35
42.66899642390817
save 35

In [ ]:
#Eval
# from infer import phonetic_embedding
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import torch.nn as nn
import torch.nn.functional as F
from phonetic_encoder import Phonetic_encoder
from acoustic_encoder import Acoustic_encoder
from linguistic_encoder import Linguistic_encoder
from char_embedding import text_to_tensor, clean_corpus
from help import wav_norm, Atention
import numpy as np
# from help import beam_search_decoding
from model import Acoustic_Phonetic_Linguistic
# from dataloader import MDD_Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
from jiwer import wer, cer
tokenizer = Wav2Vec2Processor.from_pretrained("pretrained_finetuned")

from infer import phonetic_embedding
net = Acoustic_Phonetic_Linguistic()
net.to('cuda')
net = torch.load('/content/drive/MyDrive/PL/MDD_Checkpoint/vocal_checkpoint.pth')
net.eval().to('cuda')

"""
path = '/content/37303234365f3231.wav'
lyrics = 'Trăngcôđơnlặnglẽ Nhớaiquatừng Gócphốngàyxưa Nghetrongtimvụnvỡ Baolầnnáttanvìai Cóainhớngày Xưaemơi Giờđãxatôimấtrồi Ánhtrăngsáng Lònganhemơi Giờthấmướt'
phonetic = phonetic_embedding(path).squeeze(0)
linguistic = text_to_tensor(lyrics)
linguistic = torch.tensor(linguistic)
label = text_to_tensor(dev['Lyric_path'][i])
label = torch.tensor(label)
phonetic = phonetic.to('cuda')
linguistic = linguistic.to('cuda')
phonetic = phonetic.unsqueeze(0)
linguistic = linguistic.unsqueeze(0)
outputs = net(phonetic, linguistic)"""

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


"\npath = '/content/37303234365f3231.wav'\nlyrics = 'Trăngcôđơnlặnglẽ Nhớaiquatừng Gócphốngàyxưa Nghetrongtimvụnvỡ Baolầnnáttanvìai Cóainhớngày Xưaemơi Giờđãxatôimấtrồi Ánhtrăngsáng Lònganhemơi Giờthấmướt'\nphonetic = phonetic_embedding(path).squeeze(0)\nlinguistic = text_to_tensor(lyrics)\nlinguistic = torch.tensor(linguistic)\nlabel = text_to_tensor(dev['Lyric_path'][i])\nlabel = torch.tensor(label)\nphonetic = phonetic.to('cuda')\nlinguistic = linguistic.to('cuda')\nphonetic = phonetic.unsqueeze(0)\nlinguistic = linguistic.unsqueeze(0)\noutputs = net(phonetic, linguistic)"

In [ ]:
import pandas as pd
def clean_corpus(str1):
    res1 = ""
    for i in str1:
        if i.isalpha() or i==" ":
            res1 = "".join([res1, i])
    return res1

In [ ]:
public_test = pd.read_csv("public_test.csv")
for i in range(len(public_test)):
  path = public_test['Wav_path'][i]
  path = path.split("/")[5].split(".")[0]
  lyrics = public_test['Lyric_path'][i]
  phonetic = phonetic_embedding('/content/drive/MyDrive/public_test/songs/' + path + ".wav").squeeze(0)
  # phonetic = np.load('/content/drive/MyDrive/vocal_public_test/' + path + ".npy")
  linguistic = text_to_tensor(lyrics)
  linguistic = torch.tensor(linguistic)

  phonetic = phonetic.to('cuda')
  linguistic = linguistic.to('cuda')
  phonetic = phonetic.unsqueeze(0)
  linguistic = linguistic.unsqueeze(0)
  outputs = net(phonetic, linguistic).detach().cpu().numpy()
  np.save("/content/drive/MyDrive/song_phonetic/" + path + ".npy", outputs)
  print(outputs.shape)

# path = '/content/37303234365f3231.wav'
# lyrics = 'Trăngcôđơnlặnglẽ Nhớaiquatừng Gócphốngàyxưa Nghetrongtimvụnvỡ Baolầnnáttanvìai Cóainhớngày Xưaemơi Giờđãxatôimấtrồi Ánhtrăngsáng Lònganhemơi Giờthấmướt'
# phonetic = phonetic_embedding(path).squeeze(0)
# linguistic = text_to_tensor(lyrics)
# linguistic = torch.tensor(linguistic)
# label = text_to_tensor(dev['Lyric_path'][i])
# label = torch.tensor(label)
# phonetic = phonetic.to('cuda')
# linguistic = linguistic.to('cuda')
# phonetic = phonetic.unsqueeze(0)
# linguistic = linguistic.unsqueeze(0)
# outputs = net(phonetic, linguistic)

(870, 96)
(1210, 96)
(1394, 96)
(1302, 96)
(549, 96)
(779, 96)
(638, 96)
(1296, 96)
(454, 96)
(1053, 96)
(787, 96)
(666, 96)
(539, 96)
(690, 96)
(1187, 96)
(1283, 96)
(1155, 96)
(595, 96)
(1032, 96)
(449, 96)
(1291, 96)
(1108, 96)
(1310, 96)
(1054, 96)
(1089, 96)
(1255, 96)
(923, 96)
(1271, 96)
(1138, 96)
(1140, 96)
(781, 96)
(735, 96)
(738, 96)
(489, 96)
(1119, 96)
(749, 96)
(949, 96)
(648, 96)
(863, 96)
(827, 96)
(855, 96)
(1313, 96)
(840, 96)
(979, 96)
(641, 96)
(1193, 96)
(744, 96)
(1171, 96)
(943, 96)
(1456, 96)
(667, 96)
(976, 96)
(1028, 96)
(1358, 96)
(1147, 96)
(1402, 96)
(1148, 96)
(555, 96)
(1191, 96)
(827, 96)
(509, 96)
(786, 96)
(1084, 96)
(1399, 96)
(1048, 96)
(1144, 96)
(926, 96)
(617, 96)
(538, 96)
(488, 96)
(715, 96)
(887, 96)
(1194, 96)
(810, 96)
(1390, 96)
(551, 96)
(939, 96)
(1337, 96)
(606, 96)
(1088, 96)
(891, 96)
(1154, 96)
(502, 96)
(1294, 96)
(1296, 96)
(1299, 96)
(343, 96)
(775, 96)
(1320, 96)
(1344, 96)
(1054, 96)
(1458, 96)
(1142, 96)
(1162, 96)
(1244, 96)
(1

In [13]:
#Eval
# from infer import phonetic_embedding
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import torch.nn as nn
import torch.nn.functional as F
from phonetic_encoder import Phonetic_encoder
from acoustic_encoder import Acoustic_encoder
from linguistic_encoder import Linguistic_encoder
from char_embedding import text_to_tensor, clean_corpus
from help import wav_norm, Atention
import numpy as np
# from help import beam_search_decoding
from model import Acoustic_Phonetic_Linguistic, Phonetic_Only
# from dataloader import MDD_Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
from jiwer import wer, cer
tokenizer = Wav2Vec2Processor.from_pretrained("pretrained_finetuned")

from infer import phonetic_embedding
net = Phonetic_Only()
net.to('cuda')
net = torch.load('/content/drive/MyDrive/PL/MDD_Checkpoint/no_linguistic_vocal_checkpoint.pth')
net.eval().to('cuda')

/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:375: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Phonetic_Only(
  (Phonetic_encoder): Phonetic_encoder(
    (CNN): CNN_Stack(
      (Conv2d): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (reLU): ReLU()
      (drop_out): Dropout(p=0.2, inplace=False)
      (bn): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (RNN): RNN_Stack(
      (reLU): ReLU()
      (drop_out): Dropout(p=0.2, inplace=False)
      (bn): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bilstm): LSTM(768, 384, bidirectional=True)
    )
  )
  (fc1): Linear(in_features=768, out_features=96, bias=True)
)

In [ ]:
public_test = pd.read_csv("public_test.csv")
for i in range(len(public_test)):
  path = public_test['Wav_path'][i]
  path = path.split("/")[5].split(".")[0]
  lyrics = public_test['Lyric_path'][i]
  phonetic = phonetic_embedding('/content/drive/MyDrive/vocal_public_test/' + path + ".wav").squeeze(0)


  phonetic = phonetic.to('cuda')

  phonetic = phonetic.unsqueeze(0)

  outputs = net(phonetic).detach().cpu().numpy()
  np.save("/content/drive/MyDrive/vocal_no_linguistic/" + path + ".npy", outputs)
  print(outputs.shape)


(869, 96)
(1209, 96)
(1394, 96)
(1302, 96)
(548, 96)
(778, 96)
(638, 96)
(1295, 96)
(453, 96)
(1052, 96)
(786, 96)
(666, 96)
(538, 96)
(689, 96)
(1186, 96)
(1282, 96)
(1154, 96)
(594, 96)
(1031, 96)
(449, 96)
(1290, 96)
(1108, 96)
(1309, 96)
(1053, 96)
(1088, 96)
(1254, 96)
(922, 96)
(1271, 96)
(1138, 96)
(1139, 96)
(781, 96)
(735, 96)
(738, 96)
(488, 96)
(1118, 96)
(748, 96)
(948, 96)
(648, 96)
(862, 96)
(826, 96)
(854, 96)
(1312, 96)
(839, 96)
(979, 96)
(640, 96)
(1193, 96)
(743, 96)
(1171, 96)
(942, 96)
(1456, 96)
(667, 96)
(976, 96)
(1028, 96)
(1358, 96)
(1146, 96)
(1401, 96)
(1147, 96)
(554, 96)
(1190, 96)
(827, 96)
(508, 96)
(785, 96)
(1084, 96)
(1398, 96)
(1048, 96)
(1143, 96)
(926, 96)
(617, 96)
(538, 96)
(487, 96)
(714, 96)
(886, 96)
(1193, 96)
(810, 96)
(1389, 96)
(550, 96)
(939, 96)
(1337, 96)
(605, 96)
(1087, 96)
(890, 96)
(1153, 96)
(501, 96)
(1294, 96)
(1295, 96)
(1298, 96)
(343, 96)
(774, 96)
(1319, 96)
(1343, 96)
(1053, 96)
(1457, 96)
(1142, 96)
(1161, 96)
(1244, 96)
(1

In [ ]:
outputs = outputs.detach().cpu().numpy()

In [ ]:
np.save("/content/output.npy", outputs)

In [ ]:
# /content/drive/MyDrive/output_alignment_fusion/3130303538355f3338.npy
!pip install torch
# from torch.utils.data import Dataset
# import matplotlib.pyplot as plt
# import os
# import pandas as pd
import torch.nn.functional as F
import torch
import numpy as np
# import numpy as np
# from char_embedding import tensor_to_text
import matplotlib.pyplot as plt
import glob


x = np.load("/content/output.npy")

# print(x)

x = torch.tensor(x)
predicted_ids = torch.argmax(torch.tensor(x), dim=1)


x = torch.log_softmax(x, dim=-1)
x = x.cpu().detach()


labels = ('ắ', 'ồ', 'z', 'ứ', 'ỡ', 'ì', 'x', 'ặ', 'u', 'ẹ', 'd', 'ỵ', 'r', 'p', 't', 'ỳ', 'ẩ', 'f', 'ó', 'á', 'v', 'ã', 'i', 'ư', 'ở', 'ễ', 'ụ', 'ú', 'ũ', ' ', 'ă', 'é', 'ằ', 'a', 'ấ', 'ờ', 'ữ', 'ớ', 'n', 'ý', 's', 'h', 'ơ', 'ị', 'l', 'c', 'k', 'ỷ', 'ỗ', 'ế', 'ẻ', 'ợ', 'ẫ', 'í', 'ỏ', 'ủ', 'g', 'q', 'j', 'ò', 'ỹ', 'ự', 'ô', 'b', 'y', 'ĩ', 'ỉ', 'ẵ', 'ầ', 'ê', 'ộ', 'ậ', 'm', 'ń', 'o', 'ọ', 'đ', 'ẽ', 'ử', 'à', 'è', 'e', 'ẳ', 'ổ', 'ù', 'w', 'ả', 'ạ', 'â', 'ệ', 'ề', 'õ', 'ố', 'ể', 'ừ',)

def clean_corpus(str1):
    res1 = ""
    for i in str1:
        if i.isalpha() or i==" ":
            res1 = "".join([res1, i])
    return res1.lower()
transcript = " " + 'Trăng cô đơn lặng lẽ Nhớ ai qua từng Góc phố ngày xưa Nghe trong tim vụn vỡ Bao lần nát tan vì ai Có ai nhớ ngày Xưa em ơi Giờ đã xa tôi mất rồi Ánh trăng sáng Lòng anh em ơi Giờ thấm ướt' + ""
transcript = clean_corpus(transcript)
dictionary = {c: i for i, c in enumerate(labels)}

tokens = [dictionary[c] for c in transcript]


def get_trellis(x, tokens, blank_id=95):
    num_frame = x.size(0)
    num_tokens = len(tokens)

    # Trellis has extra diemsions for both time axis and tokens.
    # The extra dim for tokens represents <SoS> (start-of-sentence)
    # The extra dim for time axis is for simplification of the code.
    trellis = torch.empty((num_frame + 1, num_tokens + 1))
    trellis[0, 0] = 0
    trellis[1:, 0] = torch.cumsum(x[:, 0], 0)
    trellis[0, -num_tokens:] = -float("inf")
    trellis[-num_tokens:, 0] = float("inf")

    for t in range(num_frame):
        trellis[t + 1, 1:] = torch.maximum(
            # Score for staying at the same token
            trellis[t, 1:] + x[t, blank_id],
            # Score for changing to the next token
            trellis[t, :-1] + x[t, tokens],
        )
    return trellis


trellis = get_trellis(x, tokens)

from dataclasses import dataclass
@dataclass
class Point:
  token_index: int
  time_index: int
  score: float


def backtrack(trellis, emission, tokens, blank_id=95):
  # Note:
  # j and t are indices for trellis, which has extra dimensions 
  # for time and tokens at the beginning.
  # When refering to time frame index `T` in trellis,
  # the corresponding index in emission is `T-1`.
  # Similarly, when refering to token index `J` in trellis,
  # the corresponding index in transcript is `J-1`.
  j = trellis.size(1) - 1
  t_start = torch.argmax(trellis[:, j]).item()

  path = []
  for t in range(t_start, 0, -1):
    # 1. Figure out if the current position was stay or change
    # Note (again):
    # `emission[J-1]` is the emission at time frame `J` of trellis dimension.
    # Score for token staying the same from time frame J-1 to T.
    stayed = trellis[t-1, j] + emission[t-1, blank_id]
    # Score for token changing from C-1 at T-1 to J at T.
    changed = trellis[t-1, j-1] + emission[t-1, tokens[j-1]]

    # 2. Store the path with frame-wise probability.
    prob = emission[t-1, tokens[j-1] if changed > stayed else 0].exp().item()
    # Return token index and time index in non-trellis coordinate.
    path.append(Point(j-1, t-1, prob))

    # 3. Update the token
    if changed > stayed:
      j -= 1
      if j == 0:
        break
  else:
    raise ValueError('Failed to align')
  return path[::-1]

path = backtrack(trellis, x, tokens)
@dataclass
class Segment:
    label: str
    start: int
    end: int
    score: float
    

    def __repr__(self):
      return f"{self.start} --> {self.end} {self.label} \n"

    @property
    def length(self):
        return self.end - self.start


def merge_repeats(path):
    i1, i2 = 0, 0
    segments = []
    while i1 < len(path):
        while i2 < len(path) and path[i1].token_index == path[i2].token_index:
            i2 += 1
        score = sum(path[k].score for k in range(i1, i2)) / (i2 - i1)
        segments.append(
            Segment(
                transcript[path[i1].token_index],
                path[i1].time_index,
                path[i2 - 1].time_index + 1,
                score,
            )
        )
        i1 = i2
    return segments


segments = merge_repeats(path)

# Merge words
def merge_words(segments, separator=" "):
    words = []
    i1, i2 = 0, 0
    while i1 < len(segments):
        if i2 >= len(segments) or segments[i2].label == separator:
            if i1 != i2:
                segs = segments[i1:i2]
                word = "".join([seg.label for seg in segs])
                score = sum(seg.score * seg.length for seg in segs) / sum(seg.length for seg in segs)
                words.append(Segment(word, segments[i1].start*20, segments[i2 - 1].end*20, score))
            i1 = i2 + 1
            i2 = i1
        else:
            i2 += 1
    return words


word_segments = merge_words(segments)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<ipython-input-45-e630c79d24bf>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_ids = torch.argmax(torch.tensor(x), dim=1)


In [ ]:
print(word_segments[0])

0.180000 --> 0.520000 trăng 



In [ ]:
print(str(word_segments[0]).split(" "))

['180.000000', '-->', '520.000000', 'trăng', '\n']


In [ ]:
def format_millis(milliseconds):
  millis = milliseconds
  millisecond = int(millis%1000)
  millisecond = str(millisecond)
  if len(millisecond)==0:
    millisecond = '000'
  elif len(millisecond) == 1:
    millisecond = '00' + millisecond
  elif len(millisecond) == 2:
    millisecond = '0' + millisecond

  seconds=(millis/1000)%60
  seconds = int(seconds)
  seconds = str(seconds)
  if len(seconds) == 1:
    seconds = '0' + seconds
  minutes=(millis/(1000*60))%60
  
  minutes = int(minutes)
  minutes = str(minutes)
  if len(minutes) == 1:
    minutes = '0' + minutes

  hours=(millis/(1000*60*60))%24
  hours = int(hours)
  hours = str(hours)
  if len(hours) == 1:
    hours = '0' + hours

  return (hours + ":" + minutes + ":" +  seconds + "," + millisecond)

In [ ]:
print(format_millis(6240))

00:00:06,240


In [ ]:
f = open("/content/output.srt", "a")

0:0:2,500


In [ ]:
for i in range(len(word_segments)):
  word_segments[i] = str(word_segments[i])
  start = word_segments[i].split(" ")[0]
  end = word_segments[i].split(" ")[2]
  word = word_segments[i].split(" ")[3]

  f.write(str(i+1) + "\n" + str(format_millis(int(start))) + " --> " + str(format_millis(int(end))) + "\n" + str(word) + "\n\n")


0:0:0,180
0:0:0,520
trăng
0:0:0,540
0:0:0,700
cô
0:0:0,840
0:0:1,40
đơn
0:0:1,160
0:0:1,340
lặng
0:0:1,460
0:0:1,640
lẽ
0:0:1,800
0:0:2,80
nhớ
0:0:2,200
0:0:2,280
ai
0:0:2,400
0:0:2,580
qua
0:0:2,720
0:0:2,980
từng
0:0:3,40
0:0:3,180
góc
0:0:3,340
0:0:3,620
phố
0:0:3,680
0:0:3,880
ngày
0:0:4,120
0:0:5,60
xưa
0:0:5,220
0:0:5,420
nghe
0:0:5,540
0:0:5,720
trong
0:0:5,880
0:0:6,140
tim
0:0:6,200
0:0:6,440
vụn
0:0:6,500
0:0:6,900
vỡ
0:0:6,980
0:0:7,340
bao
0:0:7,440
0:0:7,680
lần
0:0:7,740
0:0:7,960
nát
0:0:8,80
0:0:8,620
tan
0:0:8,720
0:0:9,0
vì
0:0:9,180
0:0:10,220
ai
0:0:10,300
0:0:10,480
có
0:0:10,740
0:0:10,840
ai
0:0:10,960
0:0:11,140
nhớ
0:0:11,220
0:0:11,380
ngày
0:0:11,520
0:0:11,720
xưa
0:0:11,840
0:0:11,960
em
0:0:12,160
0:0:12,580
ơi
0:0:12,660
0:0:12,820
giờ
0:0:12,840
0:0:13,20
đã
0:0:13,120
0:0:13,280
xa
0:0:13,360
0:0:13,580
tôi
0:0:13,640
0:0:13,800
mất
0:0:14,0
0:0:15,220
rồi
0:0:15,420
0:0:15,500
ánh
0:0:15,580
0:0:15,720
trăng
0:0:15,860
0:0:16,80
sáng
0:0:16,260
0:0:16,

In [ ]:
!pip install unidecode
import unidecode
 
def remove_accent(text):
    return unidecode.unidecode(text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 6.2 MB/s eta 0:00:00


In [ ]:
!ffmpeg -i /content/37303234365f3231.wav -filter_complex "color=c=black,subtitles=/content/output.txt[v]" -map "[v]" -map 0:a -c:a aac -shortest output.mp4

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e